# LLM product description from keywords

Giskard is an open-source framework for testing all ML models, from LLMs to tabular models. Don’t hesitate to give the project a [star on GitHub](https://github.com/Giskard-AI/giskard) ⭐️ if you find it useful!

In this notebook, you’ll learn how to create comprehensive test suites for your model in a few lines of code, thanks to Giskard’s open-source Python library.

In this example, we illustrate the procedure using **OpenAI Client** that is the default one; however, please note that our platform supports a variety of language models. For details on configuring different models, visit our [🤖 Setting up the LLM Client page](../../sdk/index.rst)

In this tutorial we will walk through a practical use case of using the Giskard LLM Scan on a Prompt Chaining task, one step at a time. Given a product name, we will ask the LLM to process 2 chained prompts using `langchain` in order to provide us with a product description. The 2 prompts can be described as follows:

1. `keywords_prompt_template`: Based on the product name (given by the user), the LLM has to provide a list of five to ten relevant keywords that would increase product visibility.
2. `product_prompt_template`: Based on the given keywords (given as a response to the first prompt), the LLM has to generate a multi-paragraph rich text product description with emojis that is creative and SEO compliant.

Use-case:

* Two-step product description generation. 1) Keywords generation -> 2) Description generation;

Outline:

* Detect vulnerabilities automatically with Giskard's scan
* Automatically generate & curate a comprehensive test suite to test your model beyond accuracy-related metrics

### Install dependencies and setup notebook

Make sure to install the `giskard[llm]` flavor of Giskard, which includes support for LLM models.
Additionally, we will install `langchain` to define the prompt templates and pipeline logic.

In [ ]:
%pip install "giskard[llm]"  --upgrade
%pip install langchain langchain-community langchain-openai --upgrade

Now, let's import all of the libraries that we will use in the notebook.

In [10]:
import os
from operator import itemgetter

import openai
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from giskard import Dataset, Model, scan


And, we set the OpenAI API key and display options.

In [ ]:
# Set the OpenAI API Key environment variable.
OPENAI_API_KEY = "..."
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Display options.
pd.set_option("display.max_colwidth", None)

Lastly, we define some of the constants that we will use in the notebook. 
Note that we are also defining two prompt templates that we will use to generate the keywords and the product description.

In [11]:
LLM_MODEL = "gpt-3.5-turbo"

TEXT_COLUMN_NAME = "product_name"

# First prompt to generate keywords related to the product name
KEYWORDS_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant that generate a CSV list of keywords related to a product name

    Example Format:
    PRODUCT NAME: product name here
    KEYWORDS: keywords separated by commas here

    Generate five to ten keywords that would increase product visibility. Begin!

    """,
        ),
        (
            "human",
            """
    PRODUCT NAME: {product_name}
    KEYWORDS:""",
        ),
    ]
)

# Second chained prompt to generate a description based on the given keywords from the first prompt
PRODUCT_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """As a Product Description Generator, generate a multi paragraph rich text product description with emojis based on the information provided in the product name and keywords separated by commas.

    Example Format:
    PRODUCT NAME: product name here
    KEYWORDS: keywords separated by commas here
    PRODUCT DESCRIPTION: product description here

    Generate a product description that is creative and SEO compliant. Emojis should be added to make product description look appealing. Begin!

    """,
        ),
        (
            "human",
            """
    PRODUCT NAME: {product_name}
    KEYWORDS: {keywords}
    PRODUCT DESCRIPTION:
        """,
        ),
    ]
)

## Detect vulnerabilities in your model

### Define a generation function

To run scans, we need to define a generation function that takes a dataframe as input and returns a list of product descriptions.
Using the prompt templates defined earlier we can create two `LLMChain` and concatenate them into a `SequentialChain` that takes as input the product name, and outputs a product description.

In [ ]:
def generation_function(df: pd.DataFrame):
    llm = ChatOpenAI(temperature=0.2, model=LLM_MODEL)

    # Define the chains.
    keywords_chain = KEYWORDS_PROMPT_TEMPLATE | llm | StrOutputParser()
    product_description_chain = (
        {"keywords": keywords_chain, "product_name": itemgetter("product_name")}
        | PRODUCT_PROMPT_TEMPLATE
        | llm
        | StrOutputParser()
    )

    return [
        product_description_chain.invoke({"product_name": product_name})
        for product_name in df["product_name"]
    ]

### Wrap generation function as Giskard Model

Before running the automatic LLM scan, we need to wrap our model into Giskard's `Model` object.

In [ ]:
# Wrap the description chain.
giskard_model = Model(
    model=generation_function,
    # A prediction function that encapsulates all the data pre-processing steps and that could be executed with the dataset
    model_type="text_generation",  # Either regression, classification or text_generation.
    name="Product keywords and description generator",  # Optional.
    description="Generate product description based on a product's name and the associated keywords."
    "Description should be using emojis and being SEO compliant.",  # Is used to generate prompts
    feature_names=["product_name"],  # Default: all columns of your dataset.
)

### Test your model with a Giskard Dataset

We can also optionally create a small dataset of queries to test that the model wrapping worked.
Let’s check that the model is correctly wrapped by running it on a small dataset of sample input prompts:

In [ ]:
# Optional: Wrap a dataframe of sample input prompts to validate the model wrapping and to narrow specific tests' queries.
corpus = [
    "Double-Sided Cooking Pan",
    "Automatic Plant Watering System",
    "Miniature Exercise Equipment",
]

giskard_dataset = Dataset(pd.DataFrame({TEXT_COLUMN_NAME: corpus}), target=None)

# Validate the wrapped model and dataset.
giskard_model.predict(giskard_dataset).prediction

### Scan your model for vulnerabilities with Giskard

We can now run Giskard's `scan` to generate an automatic report about the model vulnerabilities. This will thoroughly test different classes of model vulnerabilities, such as harmfulness, hallucination, prompt injection, etc.

The scan will use a mixture of tests from predefined set of examples, heuristics, and LLM based generations and evaluations.

Since running the whole scan can take a bit of time, let’s start by limiting the analysis to the hallucination category:

In [2]:
results = scan(giskard_model)

NameError: name 'scan' is not defined

You can save and share them with your team using the `to_html` and `to_json` methods but for now we will just display the results:

In [1]:
display(results)

NameError: name 'results' is not defined

## Generate comprehensive test suites automatically for your model

### Generate test suites from the scan

The objects produced by the scan can be used as fixtures to generate a test suite that integrates all detected vulnerabilities. Test suites allow you to evaluate and validate your model's performance, ensuring that it behaves as expected on a set of predefined test cases, and to identify any regressions or issues that might arise during development or updates.

In [35]:
test_suite = results.generate_test_suite("Test suite generated by scan")
test_suite.run()

2025-04-01 10:34:25,535 pid:64854 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'product_name': 'object'} to {'product_name': 'object'}
2025-04-01 10:34:25,537 pid:64854 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.005036
Executed 'Output plausibility' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x325333a70>, 'dataset': <giskard.datasets.base.Dataset object at 0x32cca57f0>}: 
               Test failed
               Metric: 2
               
               
2025-04-01 10:34:41,798 pid:64854 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'product_name': 'object'} to {'product_name': 'object'}
2025-04-01 10:34:41,802 pid:64854 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.009503
2025-04-01 10:34:41,808 pid:64854 MainThread giskard.datasets.base INFO     Casting dataframe colum

<TestSuiteResult (failed)>